In [9]:
import librosa
import matplotlib.pyplot as plt
import librosa.display 
import numpy as np
import sox
import os
import soundfile as sf
from itertools import groupby
import pandas as pd

In [10]:
path=r"F:\DLR\ALICE\6mALICEplus" #path to the ALICE folder
paths=r"F:\DLR\matchedspeakersound\6mplus\6mspeaker" #save the outputs in the same folder or you could create new folders

In [11]:
folders=os.listdir(path)
folders.sort()

In [12]:
def newfolder(pathin):
    folder = os.path.exists(pathin)
    if not folder:
        os.makedirs(pathin)

In [13]:
for i in range(len(folders)):
    pathtem=paths+'\\'+folders[i]
    newfolder(pathtem)

In [14]:
def outputall(path,paths,folder,folderin):
    i=0 
    with open(path+'\\'+folder+'\\'+'diarization_output.rttm', 'r', encoding='utf-8') as f: 
        lines = f.readlines()
        line_n=len(lines)
        startp=np.zeros((line_n,1))
        duration=np.zeros((line_n,1))
        who=[np.nan]*line_n
    with open(path+'\\'+folder+'\\'+'diarization_output.rttm', 'r', encoding='utf-8') as f:
        for line in f:
            line=line.split(' ')
            startp[i]=line[3] #start of an utterance
            duration[i]=line[4] #duration of an utterance
            who[i]=line[7] #who is the speaker of an utterance
            i=i+1
    endtime=startp+duration # calculate the endtime of an utterance
    startend=np.c_[startp,endtime]
    for i in range(len(startend)): # Line labeled as 'SPEECH' is confused, remove them
        if who[i]=='SPEECH':
            startend[i,:]=np.nan
    startendclear=startend[~np.isnan(startend).all(axis=1), :]
    whoclear=[]
    for i in range(len(startend)):
        if who[i]!='SPEECH':
            whoclear.append(who[i])
    conlabel=np.zeros((len(startendclear))) #initial the conversation block No.
    label=1
    for i in range(len(startendclear)):
        if i!=0:
            diff=startendclear[i,0]-startendclear[i-1,1] #latter startime-former endtime
            if (diff>=-3)&(diff<=3):
                conlabel[i]=label
            else:
                label=label+1
                conlabel[i]=label
        conlabel[0]=1
    temp=np.c_[startendclear,conlabel]
    groups = [(k, sum(1 for _ in g)) for k, g in groupby(temp[:,-1])] #If the conversation includes more than 2 (>=2) utterance, label as 1,2,3,..., or is 0
    multilabel=np.zeros((len(startendclear)))
    groupsn=np.zeros((len(groups),1))
    conchild=np.zeros((len(startendclear)))
    labelm=1
    for i in range(len(groups)):
        groupsn[i]=groups[i][1]
    for i in range(len(groupsn)):
        if groupsn[i]>=2:
            multilabel[int(np.sum(groupsn[:i])):int(np.sum(groupsn[:i+1]))]=labelm
            whot=whoclear[int(np.sum(groupsn[:i])):int(np.sum(groupsn[:i+1]))]
            if ('KCHI'in whot) or ('CHI'in whot): # if the conversation contains children's speech, it will be labeled as 1, or is 0
                conchild[int(np.sum(groupsn[:i])):int(np.sum(groupsn[:i+1]))]=1
            labelm=labelm+1
    #alls=np.c_[startendclear,whoclear,multilabel,conchild]
    alls=np.c_[startendclear,whoclear]
    allsp=pd.DataFrame(alls)
    allsp.columns=['Starttime','Endtime','Speaker']
    allsp.to_csv(paths+'\\'+folderin+'\\'+folder+'VTC.csv')

In [15]:
pathsave=r'F:\DLR\matchedspeakersound\6mplus\6mspeaker' #path to save the output csv

In [16]:
for i in range(len(folders)):
    pathpath=path+"\\"+folders[i]
    folderin=os.listdir(pathpath)
    for j in range(len(folderin)):
        outputall(pathpath,pathsave,folderin[j],folders[i])